<a href="https://colab.research.google.com/github/krahul2024/machine-learning/blob/main/projects/next_word_predict/next_word_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Imports
import tensorflow as tf, numpy as np
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
#@title Dataset
text = '''  Science very efficiently plays the role of being a faithful servant of man. In every walk of life, science is there to serve us. We require the benefits of science whether in our home, in office, in a factory, or outside.Gone are the days when only wealthy people could afford luxuries. Science has made many luxurious items of the past cheaper in price and has brought them within the reach of everybody.Computer technology is one huge benefit of science. Nowadays, it would be unimaginable to consider living without computing technology.A huge number of professions now rely totally on the computer and the internet. Besides, the computer and the internet have become our biggest source of entertainment in our everyday life.A huge number of professions now rely totally on the computer and the internet. Besides, the computer and the internet have become our biggest source of entertainment in our everyday life.Automobiles, an important scientific invention, has made our lives easy by significantly reducing everyday commuting time. The air conditioner is another scientific invention that has made our lives bearable and comfortable in the face of extreme weather conditions. Also, in the field of medical science, high-quality medicines are available that quickly remove any ailment that can happen in everyday life like headache, sprain, cough, allergy, stomach ache, fatigue etc.In spite of its tremendous benefits, there is a negative side to science. Science, unfortunately, has also done some disservice to humanity due to some of its inventions.One of the biggest harms that science has brought to humanity is in the field of armament. Although some hail the invention of gunpowder as a great achievement, humanity must rue the day when this invention happened.Steadily and relentlessly, the use and perfection of gunpowder have taken place in many new and more destructive weapons. As such, humanity now suffers due to weapons like shells, bombs, artillery, and guns. Such weapons threaten the everyday life of all individuals.Another disservice of science has been the emission of pollution. A huge amount of radioactive pollution is emitted in various parts of the world where nuclear energy production happens. Such pollution is very dangerous as it can cause cancer, radioactive sickness, and cardiovascular disease.Of course, who can ignore the massive amount of air pollution caused by automobiles, another scientific invention. Furthermore, automobiles are an everyday part of our lives that emit unimaginable levels of carbon monoxide in the air every year. Consequently, this causes various lung diseases and also contributes to global warming and acid rain.There is no doubt that science has brought about one of the greatest benefits to mankind, in spite of some of its negativities. Furthermore, science certainly has made the most impact in adding comfort to our everyday lives. As such, we must always show utmost respect to scientists for their efforts.The most important or main purpose of science is to explain the facts. Furthermore, there is no restriction in science to explain facts at random. Moreover, science systematizes the facts and builds up theories that give an explanation of such facts.My school is a special place where I spend a big part of my day. The moment I step through the school gates, I feel a sense of excitement and readiness to learn. The classrooms are filled with colorful charts and interesting books that make learning fun. My teacher is kind and helps us understand different subjects.During recess, my friends and I play in the big playground. We have swings, slides, and lots of open space to run around. I love the school library where I can read fascinating stories and discover new things. Our school also organizes events like sports day and annual functions, where we get to showcase our talents and have a great time.My school bag is always packed with notebooks, pencils, and my favorite water bottle. The school bus takes us on a journey filled with laughter and conversations with friends. I am grateful for my school because it not only teaches me academic subjects but also values and good behavior. The teachers are like guides, helping us become better individuals.My best friend is someone very special to me. We do everything together, and she is always there when I need someone to talk to or play with. We have so many fun memories together, from playing at the park to sharing our lunch during school breaks.She is like a sister to me, and we understand each other without even saying a word. We laugh together, cry together, and support each other in everything we do. My best friend has a big heart and is always ready to lend a helping hand.My family is the most important part of my life. I have my mom, dad, and a little brother. We live together, and every day is filled with love and joy. My mom takes care of us and cooks delicious meals. My dad tells us interesting stories and helps with our homework.My little brother is my playmate, and we have so much fun together. We celebrate festivals and special occasions as a family, creating beautiful memories. I feel safe and happy when I am surrounded by my family.The lotus flower is a beautiful and fascinating plant. It grows in ponds and has lovely pink or white petals. What makes the lotus special is that it grows in muddy water but remains pure and beautiful. The petals of the lotus open in the morning and close at night. The lotus is a symbol of purity and is often used in religious ceremonies. Its beauty and resilience teach us important lessons about life. I love watching lotus flowers in ponds, and they always make me feel calm and peaceful. Hello! My name is [Your Name], and I am in Class 2. I am [age] years old and live in [your city]. I have a happy family that includes my parents and siblings. My favorite things to do are [mention your hobbies or activities you enjoy]. I love going to school and learning new things. My best subject is [mention your favorite subject], and I enjoy reading books in my free time. My friends are important to me, and we have lots of fun together.In the future, I want to [mention a dream or goal you have]. I believe in working hard and being kind to others. I am excited about growing up and experiencing new adventures!My favorite food is [mention your favorite food]. It is the most delicious thing I have ever tasted! The aroma of [your favorite food] makes my mouth water, and I can't wait to eat it whenever it's on the table.Whether it's [mention different dishes or varieties], I love every bite. The flavors dance on my taste buds, and I feel happy and satisfied after a good meal of my favorite food. It's like a special treat that I look forward to! Mathematics is my favorite subject in school. I love solving problems and working with numbers. It feels like a puzzle that I enjoy figuring out. My math teacher makes the subject interesting with fun activities and games.I like how math is everywhere in our daily lives, from counting the number of toys I have to measuring ingredients for a recipe. It helps me think logically and improves my problem-solving skills. Learning math is like unlocking a secret code, and I find it exciting!My teacher is like a guiding star in my life. She is kind, patient, and always ready to help us learn. Every day, she comes to class with a smile, making learning enjoyable. She explains things in a way that is easy to understand.My teacher encourages us to ask questions and be curious about the world around us. She is not just a teacher but also a friend who cares about our well-being. I am grateful for havingsuch a wonderful teacher who inspires me to do my best.My school bag is like a treasure chest filled with everything I need for a day of learning. It has colorful notebooks, pencils, erasers, and my favorite storybook. I carry it with me everywhere, and it feels like a companion on my school journey.In my school bag, I also have my lunchbox with yummy snacks that keep me energized. It's amazing how a simple bag can hold so many things that make my school day complete. I take good care of my school bag because it carries the tools of my education.I am lucky to have wonderful neighbors. They are like an extended family, always ready to lend a helping hand. We share happy moments, celebrate festivals together, and support each other during tough times.My neighbor is a friend I can talk to and play with. We have so much fun together, whether it's in our backyard or at their house. Having good neighbors makes the place we live feel like a warm and welcoming community.My village is a peaceful and beautiful place where nature and people coexist harmoniously. Surrounded by green fields and tall trees, my village is a haven of tranquility. The air is fresh, and the sounds of birds and rustling leaves create a soothing melody.In my village, everyone knows each other, and we celebrate festivals and special occasions together. The simple and close-knit community makes living in the village a unique and joyful experience. I feel connected to the earth and the people in my village, making it a special place I am proud to call home.Spring is my favorite season of the year. It brings a burst of colors and life after the cold winter. The flowers bloom, and the trees are covered in fresh green leaves. The air is filled with the sweet fragrance of blossoms, and butterflies dance in the sunlight.I love going for walks in the park during spring, watching the flowers bloom and feeling the gentle breeze. Spring is a time of renewal and growth, and it fills my heart with happiness. The bright days and mild temperatures make it the perfect season for outdoor activities and enjoying nature's beauty.My garden is a magical place where nature comes to life. It is filled with colorful flowers, green plants, and buzzing bees. I love spending time in my garden, watching the plants grow and bloom.I have a small vegetable patch where I plant tomatoes, carrots, and other delicious vegetables. The sound of birds chirping and the scent of blooming flowers make my garden a peaceful retreat. It's a joy to see butterflies and bees visiting the flowers, creating a lively and vibrant atmosphere.Science, the pursuit of knowledge through systematic observation and experimentation, is a beacon that illuminates the mysteries of the universe. It is a vast and ever-expanding realm that touches every aspect of our lives, from the microscopic intricacies of cells to the cosmic dance of galaxies. This essay endeavors to explore the multifaceted world of science, delving into its origins, evolution, and the profound impact it has had on society.At its core, science is a relentless quest for understanding. It is not merely a collection of facts but a dynamic process that involves curiosity, investigation, and the courage to question the unknown. The roots of science trace back to ancient civilizations where early thinkers pondered the workings of the natural world. From the philosophical inquiries of the Greeks to the empirical observations of Chinese scholars, seeds of scientific inquiry were sown across cultures.The Scientific Revolution of the 16th and 17th centuries marked a pivotal moment in the history of science. Visionaries like Copernicus, Galileo, and Newton shattered age-old perceptions, propelling humanity into an era of empirical scrutiny. The heliocentric model, the laws of motion, and the universal law of gravitation laid the groundwork for a new understanding of the cosmos. Science emerged from the shadows of mysticism, embracing reason and evidence as its guiding principles.The Enlightenment further fueled the flames of scientific inquiry. Thinkers such as Bacon and Descartes championed the scientific method—a systematic approach involving observation, experimentation, and analysis. This method became the cornerstone of scientific investigation, providing a structured pathway to unravel the secrets of the natural world. As scientific knowledge expanded, so did the realization of its transformative power.In the 19th century, the rise of industrialization and technological advancements ushered in an era of unprecedented scientific discovery. Darwin's theory of evolution reshaped our understanding of life's diversity, while Maxwell's equations illuminated the nature of electromagnetism. The periodic table, a testament to Mendeleev's ingenuity, organized the building blocks of matter. As steam engines roared to life and telegraph wires spanned continents, science became an integral force shaping the modern world.The 20th century witnessed an explosion of scientific breakthroughs that transformed society on an unprecedented scale. Einstein's theory of relativity revolutionized our understanding of space and time, leading to technological marvels like GPS. The discovery of DNA's structure by Watson and Crick unveiled the genetic code, laying the foundation for modern genetics and biotechnology. The advent of quantum mechanics opened a realm of possibilities at the subatomic level, challenging our perceptions of reality.Science's impact on medicine has been particularly profound. The development of antibiotics revolutionized healthcare, saving countless lives. Vaccines, a triumph of medical science, eradicated deadly diseases and fortified humanity against pandemics. Advances in imaging technology, from X-rays to MRI, have enabled physicians to peer inside the human body with unprecedented clarity, facilitating early diagnosis and precise treatments.Space exploration stands as a testament to humanity's audacious spirit of inquiry. The race to the moon in the mid-20th century captured the world's imagination, showcasing the power of science to transcend earthly boundaries. Today, rovers explore the Martian surface, telescopes peer into distant galaxies, and satellites monitor Earth's climate. Each discovery expands the frontiers of our knowledge and fuels our aspirations to venture further into the cosmos.The interconnectedness of scientific disciplines is a defining feature of contemporary research. The convergence of biology, chemistry, and physics has given rise to interdisciplinary fields such as biochemistry and biophysics. The marriage of computer science and genetics has birthed the field of bioinformatics, revolutionizing our ability to analyze vast genomic datasets. Collaborations across borders and disciplines have become essential to tackling complex challenges, from climate change to global health crises.Yet, for all its triumphs, science is not without ethical and moral considerations. The same technologies that alleviate suffering can also be wielded for destructive purposes. The ethical implications of genetic engineering, artificial intelligence, and nuclear technologies demand careful reflection and responsible governance. As science pushes the boundaries of what is possible, society grapples with the responsibility to ensure these advances are harnessed for the greater good.Education plays a pivotal role in fostering a scientific mindset. The cultivation of curiosity, critical thinking, and a passion for discovery begins in classrooms around the world. Science education goes beyond memorization of facts; it nurtures the spirit of inquiry and empowers individuals to engage with the world as informed and thoughtful citizens. In an era of information overload, the ability to discern reliable evidence from misinformation becomes a crucial skill, emphasizing the role of science literacy in shaping a well-informed society.The democratization of scientific knowledge in the digital age has brought both opportunities and challenges. The internet serves as a vast repository of information, enabling access to scientific literature, educational resources, and collaborative platforms. However, the proliferation of pseudoscience and misinformation also poses a threat, requiring vigilant efforts to promote scientific literacy and critical thinking.As we stand on the threshold of the future, the trajectory of science appears boundless. From unlocking the secrets of the human brain to harnessing the potential of renewable energy, the challenges and possibilities are immense. The pursuit of scientific knowledge is not a solitary endeavor but a collective journey that transcends borders and cultures. In the grand tapestry of human history, science is the thread that weaves together the narrative of exploration, discovery, and understanding.In conclusion, science is a dynamic and transformative force that has shaped the course of human history. From ancient philosophical musings to modern-day technological marvels, the journey of science is marked by curiosity, inquiry, and the unquenchable thirst for knowledge. As we navigate the complexities of the 21st century, the torch of scientific inquiry continues to illuminate the path ahead, guiding humanity toward a future where the wonders of the universe are unveiled, one discovery at a time.In the intricate tapestry of life on Earth, animals stand as vibrant threads, weaving together the story of biodiversity and adaptation. From the microscopic realms of single-celled organisms to the majestic presence of apex predators, the animal kingdom encompasses an astounding array of species, each playing a unique role in the intricate dance of ecosystems. This essay embarks on a journey to explore the multifaceted world of animals, their evolutionary marvels, ecological significance, and the delicate balance that sustains life on our planet.The story of animals unfolds over millions of years, a tale of evolutionary ingenuity and adaptation. The first multicellular animals emerged in Earth's oceans over 600 million years ago, paving the way for the breathtaking diversity we witness today. From the earliest arthropods to the complex vertebrates that grace land, sea, and air, evolution has sculpted a staggering variety of forms, each finely tuned to its environment.The process of natural selection has driven the development of specialized adaptations, enabling animals to thrive in diverse habitats. From the camouflage of chameleons to the swift wings of hummingbirds, these adaptations reflect the ingenious responses of animals to the challenges posed by their surroundings. Evolutionary arms races between predators and prey have led to the development of extraordinary traits, from the speed of cheetahs to the stealth of predators like the mantis shrimp.Animals play pivotal roles in maintaining the delicate balance of ecosystems. They serve as pollinators, seed dispersers, and decomposers, contributing to the functioning of ecosystems that support all life. The intricate web of relationships between animals and their environments highlights the interdependence that characterizes the natural world.Predator-prey interactions regulate populations and prevent unchecked growth that could destabilize ecosystems. The migrations of wildebeests in Africa, the pollination services of bees, and the intricate dance of predator and prey in marine ecosystems exemplify the interconnectedness of life. The loss of key species can have cascading effects, impacting the stability and resilience of entire ecosystems—a phenomenon known as trophic cascades.Biodiversity, the variety of life on Earth, is a testament to the resilience and adaptability of animals. Rainforests, coral reefs, and grasslands harbor a wealth of species, each contributing to the intricate mosaic of life. The loss of biodiversity, driven by factors such as habitat destruction, climate change, and overexploitation, poses a threat not only to individual species but to the functioning of ecosystems and the services they provide.Conservation efforts strive to protect endangered species, restore habitats, and mitigate the impacts of human activities on wildlife. Zoos, wildlife reserves, and protected areas serve as havens for species on the brink of extinction, providing a chance for recovery and eventual reintroduction into the wild. Global initiatives aim to address the root causes of biodiversity loss and foster a harmonious coexistence between humans and the animal kingdom.Beyond their ecological roles, animals exhibit remarkable cognitive abilities and complex social structures. From the problem-solving skills of corvids to the intricate communication of dolphins, these traits challenge traditional notions of animal intelligence. Social animals, such as elephants and wolves, engage in cooperative behaviors, forming familial bonds and intricate social hierarchies.The study of animal behavior sheds light on the rich tapestry of emotions and social dynamics that shape their lives. Elephants mourn their deceased companions, wolves work collaboratively to hunt, and dolphins engage in playful activities that foster social bonds. The recognition of sentience in animals underscores the ethical considerations surrounding their treatment and welfare.The relationship between humans and animals extends beyond the wild realms. Domestication, a process spanning millennia, has led to the emergence of domesticated animals that play integral roles in human societies. Dogs, the first domesticated species, have evolved from working companions to beloved family members, showcasing the transformative power of this interspecies bond.Animals offer companionship, emotional support, and even therapeutic benefits to humans. The therapeutic use of horses in equine-assisted therapy, the comforting presence of therapy dogs, and the role of animals in mitigating stress highlight the multifaceted connections that can exist between humans and animals.Despite the marvels of the animal kingdom, it faces unprecedented challenges in the Anthropocene—the current epoch marked by human influence. Habitat destruction, pollution, climate change, and poaching pose existential threats to many species. The ethical considerations surrounding animal welfare, conservation, and the use of animals in research prompt reflection on humanity's responsibilities as stewards of the planet.Striking a balance between the needs of human societies and the well-being of animals requires thoughtful consideration and sustainable practices. Ethical farming, wildlife conservation, and responsible tourism are avenues through which society can contribute to the preservation of the animal kingdom.In conclusion, animals embody the diversity, resilience, and interconnectedness of life on Earth. Their evolutionary journeys, ecological roles, and complex behaviors enrich the fabric of our planet. As stewards of this shared home,humans bear the responsibility to protect and cherish the animal kingdom, recognizing its intrinsic value and the integral role it plays in sustaining the web of life. Through a harmonious coexistence, we can ensure that future generations inherit a world where the wondrous tapestry of animals continues to thrive.Travelling, the act of journeying from one place to another, is a timeless human endeavor that satisfies our innate curiosity and thirst for discovery. Whether by foot, car, train, or plane, the desire to explore new horizons and experience different cultures is a universal aspect of the human experience. This essay embarks on a simple exploration of travelling, highlighting its joys, challenges, and the profound impact it has on individuals and societies.At its core, travelling is about venturing beyond the familiar, stepping into the unknown, and embracing the richness of diversity. It could be a leisurely stroll through a nearby park, a road trip to a neighboring town, or an international adventure across continents. The essence of travelling lies not only in the physical distance covered but in the mental and emotional landscapes traversed.One of the most enriching aspects of travelling is the opportunity to encounter new cultures. From savoring local cuisine to immersing oneself in traditional festivities, every journey unfolds a tapestry of customs and traditions. The act of breaking bread with strangers, learning a few phrases in a foreign language, and appreciating local customs fosters a sense of connection that transcends geographical boundaries.Travelling often leads us to the embrace of nature's wonders. From serene landscapes to breathtaking vistas, nature becomes a companion on the journey. Whether it's the rustle of leaves in a forest, the rhythmic crashing of waves on a beach, or the panoramic views from a mountain summit, the natural world provides a backdrop of beauty and tranquility that rejuvenates the spirit.While travelling brings joy and discovery, it also comes with its share of challenges. From navigating unfamiliar terrain to facing language barriers, each obstacle becomes an opportunity for growth and adaptation. The resilience developed through overcoming challenges becomes a lasting souvenir of the journey.Travelling opens the door to forging human connections. Whether it's sharing stories with fellow travelers, seeking guidance from locals, or forming bonds with hosts, the threads of human connection weave a narrative of shared experiences. These connections, often fleeting yet profound, remind us of the interconnectedness that unites humanity.Every journey leaves behind a trail of memories and souvenirs. A photograph capturing a sunset, a seashell collected from a distant shore, or the taste of a unique dish—all become tangible reminders of the experiences lived. These mementos serve as windows into the past, allowing one to relive the journey in the quiet moments of reflection.Travelling is not merely about external exploration but also an inward journey of self-discovery. The solitude of a mountain hike, the contemplation by a flowing river, or the quiet moments in a bustling city—all provide opportunities for introspection and self-awareness. Each journey becomes a chapter in the ongoing narrative of personal growth.On a broader scale, travelling contributes to the cultural exchange between societies. The influx of diverse perspectives fosters tolerance and understanding, breaking down stereotypes and prejudices. The tourism industry becomes a bridge that connects people and economies, fostering mutual appreciation and cooperation.As the journey unfolds, there is a unique joy in returning home. The familiar sights, the comfort of one's own bed, and the embrace of loved ones create a sense of grounding. The experiences gained while travelling become a part of the tapestry of one's identity, enriching the understanding of oneself and the world.In conclusion, travelling is a celebration of the human spirit's quest for exploration and connection. Whether a small excursion or a grand adventure, each journey contributes to the collective story of humanity. Through the simple act of travelling, individuals and societies continue to weave a tapestry of experiences, creating a mosaic that reflects the beauty of diversity and the shared journey of life. '''

len(text)

27020

In [3]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])

words = tokenizer.word_index
vocab_size = len(words) + 1
len(words), vocab_size

(1464, 1465)

In [4]:
import re
split_pattern = r'[.,?:;"()]'
sequences = re.split(split_pattern, text)
sequences = [seq for seq in sequences if seq] # remove duplicates
len(sequences)

524

In [5]:
# convert the sentences into tokens
input_seq = []

for seq in sequences :
  tokens = tokenizer.texts_to_sequences([seq])[0]
  for i in range(1, len(tokens)):
    input_seq.append(tokens[:i+1])

len(input_seq), input_seq[0]

(3727, [11, 175])

In [6]:
max_len = max([len(seq) for seq in input_seq])  # get maximumn length sequence
print(max_len)

# pad the input_sequence with necessary values as to make all the inputs of constant size
input_seq = np.array(pad_sequences(input_seq,  maxlen = max_len, padding = 'pre'))
input_seq.shape, input_seq[0]

22


((3727, 22),
 array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,  11, 175], dtype=int32))

In [7]:
x, y= input_seq[:, :-1], input_seq[:,-1]
y = to_categorical(y, num_classes = vocab_size)
x.shape, y.shape, y[0]

((3727, 21), (3727, 1465), array([0., 0., 0., ..., 0., 0., 0.], dtype=float32))

In [ ]:
# create the model
model = Sequential([
    Embedding(
        vocab_size, 100, input_length = x.shape[1]
    ),
    LSTM(100),
    Dense(
        vocab_size, activation = 'softmax'
    )
])

model.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

model.summary()

In [45]:
history = model.fit(x, y, epochs = 150)

Epoch 1/150
117/117 [==============================] - 3s 28ms/step - loss: 0.5182 - accuracy: 0.8787
Epoch 2/150
117/117 [==============================] - 3s 28ms/step - loss: 0.5113 - accuracy: 0.8755
Epoch 3/150
117/117 [==============================] - 5s 44ms/step - loss: 0.5020 - accuracy: 0.8803
Epoch 4/150
117/117 [==============================] - 3s 28ms/step - loss: 0.4938 - accuracy: 0.8809
Epoch 5/150
117/117 [==============================] - 3s 28ms/step - loss: 0.4872 - accuracy: 0.8793
Epoch 6/150
117/117 [==============================] - 3s 27ms/step - loss: 0.4806 - accuracy: 0.8787
Epoch 7/150
117/117 [==============================] - 5s 44ms/step - loss: 0.4730 - accuracy: 0.8830
Epoch 8/150
117/117 [==============================] - 3s 27ms/step - loss: 0.4657 - accuracy: 0.8825
Epoch 9/150
117/117 [==============================] - 4s 31ms/step - loss: 0.4600 - accuracy: 0.8841
Epoch 10/150
117/117 [==============================] - 5s 46ms/step - loss: 0.453

In [46]:
model.save('word_predict.keras')

In [47]:
def predict_word(sequence):
    tokens = tokenizer.texts_to_sequences([sequence])[0]
    input_sequence = np.array(pad_sequences([tokens], maxlen = max_len-1, padding = 'pre'))
    pred = np.argmax(model.predict(input_sequence, verbose = 0))
    # print(pred)
    for key in words :
        if(words[key] == pred) :
            return key

predict_word('what ')

'makes'

In [37]:
#@title Test Cases
test_cases = [
    "I",
    "The",
    "She",
    "My",
    "Reading",
    "Tomorrow",
    "Coffee",
    "Music",
    "In",
    "I",
    "My",
    "Pizza",
    "After",
    "The",
    "Nature",
    "I",
    "Every",
    "Learning",
    "When",
    "My",
    "To",
    "On",
    "Creativity",
    "The",
    "The",
    "I",
    "The",
    "My",
    "A",
    "I am",
    "The sun",
    "She always",
    "My favorite",
    "Reading a",
    "Tomorrow, I",
    "Coffee tastes",
    "Music helps",
    "In the",
    "I enjoy",
    "My pet",
    "Pizza topped",
    "After work,",
    "The secret",
    "Nature is",
    "I feel",
    "Every morning,",
    "Learning a new",
    "When it",
    "My favorite",
    "To stay",
    "On weekends,",
    "Creativity is",
    "The key to",
    "The more",
    "I believe",
    "The ocean",
    "My favorite",
    "A smile can",
]

In [48]:
for test in test_cases :
    print(test , '--->', predict_word(test))

I ---> love
The ---> air
She ---> is
My ---> best
Reading ---> plants
Tomorrow ---> a
Coffee ---> a
Music ---> a
In ---> the
I ---> love
My ---> best
Pizza ---> a
After ---> a
The ---> air
Nature ---> becomes
I ---> love
Every ---> journey
Learning ---> a
When ---> we
My ---> best
To ---> we
On ---> a
Creativity ---> a
The ---> air
The ---> air
I ---> love
The ---> air
My ---> best
A ---> huge
I am ---> grateful
The sun ---> air
She always ---> makes
My favorite ---> things
Reading a ---> great
Tomorrow, I ---> love
Coffee tastes ---> a
Music helps ---> us
In the ---> grand
I enjoy ---> reading
My pet ---> best
Pizza topped ---> a
After work, ---> as
The secret ---> code
Nature is ---> a
I feel ---> connected
Every morning, ---> and
Learning a new ---> things
When it ---> grows
My favorite ---> things
To stay ---> we
On weekends, ---> a
Creativity is ---> a
The key to ---> the
The more ---> of
I believe ---> in
The ocean ---> air
My favorite ---> things
A smile can ---> illuminates
